# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

Загрузим данные о скважинах в трёх регионах. Данные содержат:
* id — уникальный идентификатор скважины;
* f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
* product — объём запасов в скважине (тыс. баррелей).

In [2]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

In [3]:
data_a = pd.read_csv('/datasets/geo_data_0.csv')
data_b = pd.read_csv('/datasets/geo_data_1.csv')
data_c = pd.read_csv('/datasets/geo_data_2.csv')

data = {}
counter = 65

for df in [data_a, data_b, data_c]:
    key = chr(counter)
    value = df
    data[key] = value
    counter += 1
    
regions = list(data.keys())

print(regions)

['A', 'B', 'C']


In [4]:
data_a.describe().T

,count,mean,std,min,25%,50%,75%,max
f0,100000.0,0.500419,0.871832,-1.408605,-0.072580,0.502360,1.073581,2.362331
f1,100000.0,0.250143,0.504433,-0.848218,-0.200881,0.250252,0.700646,1.343769
f2,100000.0,2.502647,3.248248,-12.088328,0.287748,2.515969,4.715088,16.003790
product,100000.0,92.500000,44.288691,0.000000,56.497507,91.849972,128.564089,185.364347


In [5]:
data_b.describe().T

,count,mean,std,min,25%,50%,75%,max
f0,100000.0,1.141296,8.965932,-31.609576,-6.298551,1.153055,8.621015,29.421755
f1,100000.0,-4.796579,5.119872,-26.358598,-8.267985,-4.813172,-1.332816,18.734063
f2,100000.0,2.494541,1.703572,-0.018144,1.000021,2.011479,3.999904,5.019721
product,100000.0,68.825000,45.944423,0.000000,26.953261,57.085625,107.813044,137.945408


In [6]:
data_c.describe().T

,count,mean,std,min,25%,50%,75%,max
f0,100000.0,0.002023,1.732045,-8.760004,-1.162288,0.009424,1.158535,7.238262
f1,100000.0,-0.002081,1.730417,-7.084020,-1.174820,-0.009482,1.163678,7.844801
f2,100000.0,2.495128,3.473445,-11.970335,0.130359,2.484236,4.858794,16.739402
product,100000.0,95.000000,44.749921,0.000000,59.450441,94.925613,130.595027,190.029838


Осмотрены входные данные, они не содержат дупликатов и пропусков, и готовы к дальнейшему анализу. Также была изучена корреляция параметров влияющих на таргет. Для разных регионов различные признаки имеют максимальную корреляцию:

* Для региона А - f2 (положительно) и f1 (отрицательно) 
* Для региона В - f1
* Для региона С - f2

Для выполнения дальнейшего исследования нам надо обучить алгоритм линейной регрессии для построения модели. Для этого разделим данные на тренировочные и валидационные выборки, в соотношении 75% / 25%.

In [6]:
for r in regions:
    data[r + '_train'], data[r + '_valid'] = train_test_split(data[r], test_size=0.25, random_state=12345)
    
    data['features_' + r + '_train'] = data[r + '_train'].drop(['product', 'id'], axis=1)
    data['target_' + r + '_train'] = data[r + '_train']['product']
    data['features_' + r + '_valid'] = data[r + '_valid'].drop(['product', 'id'], axis=1)
    data['target_' + r + '_valid'] = data[r + '_valid']['product']    

## Обучение и проверка модели

Проведём обучение модели на тренировочной выборки для 3 регионов. Данные предсказаные моделью добавим новым столбцом в датафрейм валидационной выборки.

In [7]:
for r in regions:
       
    model = LinearRegression()
    model.fit(data['features_' + r + '_train'], data['target_' + r + '_train'])
    
    data[r + '_valid']['predictions'] = model.predict(data['features_' + r + '_valid'])
    
    print('Средний предсказанный запас скважины региона А - {:.3f}'.format(data[r + '_valid']['predictions'].mean()))
    print('RMSE {:.3f}'.format(mean_squared_error(data[r + '_valid']['product'], data[r + '_valid']['predictions']) ** 0.5))
    print('Фактический запас - {:.3f}'.format(data[r + '_valid']['product'].mean()))

Средний предсказанный запас скважины региона А - 92.593
RMSE 37.579
Фактический запас - 92.079
Средний предсказанный запас скважины региона А - 68.729
RMSE 0.893
Фактический запас - 68.723
Средний предсказанный запас скважины региона А - 94.965
RMSE 40.030
Фактический запас - 94.884


Средние величины запасов нефти скважины предсказанные моделью и фактические, довольно близки. Проведенные расчеты метрики RMSE показывает, что более точно модель предсказывает запасы нефти в регионе B. Можно утверждать, что модель корректна и адекватна для решения задачи.

## Подготовка к расчёту прибыли

Для проведения расчетов прибыли установим:
* цены барраля нефти 450 ₽
* бюджет разработки нефти в новом регионе 10 000 000 000 ₽

In [8]:
BARREL_PRICE = 450_000
WELL_COST = 10_000_000_000
MIN_STORAGE = WELL_COST / BARREL_PRICE
WELLS = 500
WELLS_TOP = 200

print('Объем запаса нефти необходимый для окупаемости разработки в регионе: {:.0f} тысяч баррелей'.format(MIN_STORAGE))
print('Объем запаса нефти необходимый для окупаемости разработки одной скважины из 200 в регионе: {:.0f} тысяч баррелей'.format(MIN_STORAGE/200))
for r in regions:
    print('Количество скважин для окупаемости разработки нефти в регионе А: {:.0f}'.format(MIN_STORAGE / (data[r + '_valid']['predictions'].mean()) ))


Объем запаса нефти необходимый для окупаемости разработки в регионе: 22222 тысяч баррелей
Объем запаса нефти необходимый для окупаемости разработки одной скважины из 200 в регионе: 111 тысяч баррелей
Количество скважин для окупаемости разработки нефти в регионе А: 240
Количество скважин для окупаемости разработки нефти в регионе А: 323
Количество скважин для окупаемости разработки нефти в регионе А: 234


Проведены расчеты объема запаса нефти для окупаемости разработки новых скважин в регионе и составил 22 222 тысячи баррелей. 

С учетом среднего значения запаса нефти в скважинах и величиной объема запаса нефти для окупаемости разработки новых скважин в регионе расчитанно количество скважин для окупаемости.

С учетом средних значений запаса нефти, в каждом регионе необходимо бурить более 200 скважин, чтобы окупиться.

## Расчёт прибыли и рисков 

In [9]:
def profit(storage, b, w):
    return (storage*b) - w

In [10]:
state = np.random.RandomState(12345)
def region_recon(region, wells, wells_top):
    values = []
    for i in range(1000):
        subsample = data[r+'_valid'].sample(n=wells, replace=True, random_state=state)
        subsample_sorted = subsample.sort_values(by='predictions', ascending=False)[:wells_top]
        storage = subsample_sorted['product'].sum()
        values.append(profit(storage, BARREL_PRICE, WELL_COST))

    values = pd.Series(values)
    
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    mean = values.mean()
    risk = values[values<0].count()/values.count()
    
    print('Регион {}'.format(region))
    print('Средняя прибыль в регионе - ₽ {:,.2f}'.format(mean))
    print('Доверительный интервал равен - [ ₽ {:,.2f}, ₽ {:,.2f} ]'.format(lower, upper))
    print('Риск понести убытки составляет - {:.2%}'.format(risk))

for r in regions:
    region_recon(r, WELLS, WELLS_TOP)

Регион A
Средняя прибыль в регионе - ₽ 396,164,984.80
Доверительный интервал равен - [ ₽ -111,215,545.89, ₽ 909,766,941.55 ]
Риск понести убытки составляет - 6.90%
Регион B
Средняя прибыль в регионе - ₽ 461,155,817.28
Доверительный интервал равен - [ ₽ 78,050,810.75, ₽ 862,952,060.26 ]
Риск понести убытки составляет - 0.70%
Регион C
Средняя прибыль в регионе - ₽ 392,950,475.17
Доверительный интервал равен - [ ₽ -112,227,625.38, ₽ 934,562,914.55 ]
Риск понести убытки составляет - 6.50%


## Выводы

Проведены расчеты прибыли нефтедобычи в различных регионах, а также были проведены оценки рисков получить отрицательную прибыль.
По полученным результатам регион В является максимально выгодным для разработки нефти ввиду низкого риска получить отрицательную прибыль.